In [3]:
import tensorflow as tf
from utils.modelLoader import ModelLoader
import pandas as pd
from utils.utilities import buildRunName
import numpy as np


In [2]:
@tf.function
def decode_img(img_path):
    """
    function read image from filepath and format it into a tensor
    :param img_path: filepath of the image
    :return: decodes image as tensor
    """
    image_size = (224, 224)
    num_channels = 3
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(
        img, channels=num_channels, expand_animations=False
    )
    img = tf.image.resize(img, image_size, method="bilinear")
    img.set_shape((image_size[0], image_size[1], num_channels))
    return img

def process_path(file_path, labels):
    label = {'out_age_prediction': tf.reshape(tf.keras.backend.cast(labels[0], tf.keras.backend.floatx()), (1, 1)),
             'out_face_detection': tf.reshape(tf.keras.backend.cast(labels[1], tf.keras.backend.floatx()), (1, 1)),
             'out_mask_detection': tf.reshape(tf.keras.backend.cast(labels[2], tf.keras.backend.floatx()), (1, 1))}
    img = decode_img(file_path)
    return img, label

def group_ages(age: int):
    current_range = [
        ( 0, 0),
        ( 1,10),
        (11,20),
        (21,30),
        (31,40),
        (41,50),
        (51,60),
        (61,70),
        (71,80),
        (81,90),
        (91,100)
    ]
    if isinstance(age, int) and age >= current_range[0] and age <= current_range[1]:
        return current_range.index(current_range)
    else:
        return age

def create_dataset(data):
    data = tf.data.Dataset.from_tensor_slices(
        (data["Filepath"], data[["Age", "Face", "Mask"]]))
    ds = data.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(32)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds


metaData = pd.read_json("../data_meta/meta_all_cropped.json")
metaData = metaData.sample(frac=1, random_state=123).reset_index(drop=True)
metaData_train = metaData.iloc[0:int(metaData.__len__() * 0.7)]
metaData_val =  metaData.iloc[int(metaData.__len__() * 0.7)+1:int(metaData.__len__() * 0.85)]
metaData_test = metaData.iloc[int(metaData.__len__() * 0.85)+1:]

train_ds = create_dataset(metaData_train)
val_ds = create_dataset(metaData_val)
test_ds = create_dataset(metaData_test)

model = ModelLoader().loadMobileNetV1Multi(10)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "out_age_prediction": tf.keras.losses.SparseCategoricalCrossentropy(ignore_class=-1),
        "out_face_detection": tf.keras.losses.BinaryCrossentropy(),
        "out_mask_detection": tf.keras.losses.BinaryCrossentropy(),
    },
    loss_weights={
        "out_age_prediction": 0.33,
        "out_face_detection": 0.33,
        "out_mask_detection": 0.33,
    },
    metrics={
        "out_age_prediction": tf.keras.metrics.SparseCategoricalAccuracy(),
        "out_face_detection": tf.keras.metrics.BinaryAccuracy(),
        "out_mask_detection": tf.keras.metrics.BinaryAccuracy(),
    },
)


log_dir = "../logs/fit/" + buildRunName("MobileNet_MultiTask", 50, 32)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[tensorboard_callback]
)

model.save("../models/" + buildRunName("MobileNet_MultiTask", 50, 32))

Epoch 1/50
866/866 [==============================] - 63s 69ms/step - loss: 0.5495 - out_face_detection_loss: 0.0654 - out_mask_detection_loss: 0.0442 - out_age_prediction_loss: 1.5555 - out_face_detection_binary_accuracy: 0.9772 - out_mask_detection_binary_accuracy: 0.9858 - out_age_prediction_sparse_categorical_accuracy: 0.2238 - val_loss: 0.4620 - val_out_face_detection_loss: 0.0079 - val_out_mask_detection_loss: 0.0155 - val_out_age_prediction_loss: 1.3765 - val_out_face_detection_binary_accuracy: 0.9985 - val_out_mask_detection_binary_accuracy: 0.9953 - val_out_age_prediction_sparse_categorical_accuracy: 0.2686
Epoch 2/50
866/866 [==============================] - 58s 67ms/step - loss: 0.4653 - out_face_detection_loss: 0.0143 - out_mask_detection_loss: 0.0165 - out_age_prediction_loss: 1.3791 - out_face_detection_binary_accuracy: 0.9954 - out_mask_detection_binary_accuracy: 0.9943 - out_age_prediction_sparse_categorical_accuracy: 0.2497 - val_loss: 0.4478 - val_out_face_detection_

INFO:tensorflow:Assets written to: ../models/MobileNet_MultiTask_epochs-50_batch-32\assets


INFO:tensorflow:Assets written to: ../models/MobileNet_MultiTask_epochs-50_batch-32\assets


In [3]:
model.evaluate(test_ds)

186/186 [==============================] - 10s 52ms/step - loss: 0.8817 - out_face_detection_loss: 0.0052 - out_mask_detection_loss: 0.0249 - out_age_prediction_loss: 2.6418 - out_face_detection_binary_accuracy: 0.9988 - out_mask_detection_binary_accuracy: 0.9981 - out_age_prediction_sparse_categorical_accuracy: 0.2472


[0.8817152976989746,
 0.005206682253628969,
 0.024870164692401886,
 2.64178729057312,
 0.9988203644752502,
 0.9981462955474854,
 0.24721941351890564]

image = tf.io.read_file("../data/age/99_1_0_20170113013141679.jpg")
image = tf.image.decode_image(image, channels=3)
image = np.expand_dims(image.numpy(), axis=0)
image = tf.image.resize(image, (224,224))
model.predict(image)

metaData = pd.read_json("../data_meta/meta_all.json")

dataset = tf.data.Dataset.from_tensor_slices((metaData["Filepath"], metaData[["Face", "Mask", "Age"]]))

def _parse_function(filename, labels:tf.Tensor):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(
        image, channels=3
    )  # Channels needed because some test images are b/w
    image = tf.image.resize(image, (224, 224))
    image.set_shape((224, 224, 3))
    
    labelsDict = dict()
    try:
        labelsDict =  {'out_age_prediction': tf.reshape(tf.keras.backend.cast(labels[0], tf.keras.backend.floatx()), (-1, 1)),
            'out_face_detection': tf.reshape(tf.keras.backend.cast(labels[1], tf.keras.backend.floatx()), (-1, 1)),
            'out_mask_detection': tf.reshape(tf.keras.backend.cast(labels[2], tf.keras.backend.floatx()), (-1, 1))}
    except:
        pass
    print(labelsDict)
    return image, labelsDict


dataset = dataset.map(_parse_function)

dataset_train = dataset.take(dataset.__len__().numpy() * 0.8)
dataset_val = dataset.skip(dataset.__len__().numpy() * 0.8).take(dataset.__len__().numpy() * 0.2)
dataset_train = dataset_train.batch(32)
dataset_val = dataset_val.batch(32)

# train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1.0 / 255, validation_split=0.2
# )

# train_images = train_generator.flow_from_dataframe(
#     dataframe=metaData,
#     x_col="Filepath",
#     y_col=["Face", "Mask", "Age"],
#     target_size=(224, 224),
#     color_mode="rgb",
#     class_mode="raw",
#     batch_size=32,
#     shuffle=True,
#     seed=123,
#     subset="training",
# )


# val_images = train_generator.flow_from_dataframe(
#     dataframe=metaData,
#     x_col="Filepath",
#     y_col=["Face", "Mask", "Age"],
#     target_size=(224, 224),
#     color_mode="rgb",
#     class_mode="raw",
#     batch_size=32,
#     shuffle=True,
#     seed=123,
#     subset="validation",
# )

model: tf.keras.Model = ModelLoader().loadMobileNetV1Multi(11)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "out_age_prediction": tf.keras.losses.SparseCategoricalCrossentropy(
            ignore_class=-1
        ),
        "out_face_detection": tf.keras.losses.BinaryCrossentropy(),
        "out_mask_detection": tf.keras.losses.BinaryCrossentropy(),
    },
    loss_weights={'out_face_detection': 0.33,
                    'out_mask_detection': 0.33,
                    'out_age_prediction': 0.33},
    metrics={
        "out_age_prediction": [tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanSquaredError()],
        "out_face_detection": tf.keras.metrics.Accuracy(),
        "out_mask_detection": tf.keras.metrics.Accuracy(),
    },
)

log_dir = "../logs/fit/" + buildRunName("MobileNet_Multi_Dropout-20", 10, 32)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=10,
    callbacks=[tensorboard_callback],
)

model.save("../models/" + buildRunName("MobileNet_Multi_Dropout-20", 10, 32))
